# Convolutional Neural Network

### Importing the libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255, # feature scaling to each of the pixel
    shear_range=0.2, # these 3 lines are for transformation
    zoom_range=0.2,
    horizontal_flip=True
    )

training_set = train_datagen.flow_from_directory(
        'datasets/cnn-dataset/training_set',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 64x64
        batch_size=32,
        class_mode='binary' # since we use binary_crossentropy loss, we need binary labels
    )  


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
        'datasets/cnn-dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary'
    )

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',input_shape=[64,64,3]))

c:\Users\xnoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data=test_set, epochs=25)

c:\Users\xnoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 657ms/step - accuracy: 0.5502 - loss: 0.6846

c:\Users\xnoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 205s 814ms/step - accuracy: 0.5503 - loss: 0.6845 - val_accuracy: 0.6700 - val_loss: 0.6111
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 71ms/step - accuracy: 0.6601 - loss: 0.6207 - val_accuracy: 0.7155 - val_loss: 0.5653
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 74ms/step - accuracy: 0.7163 - loss: 0.5587 - val_accuracy: 0.7440 - val_loss: 0.5357
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 69ms/step - accuracy: 0.7357 - loss: 0.5345 - val_accuracy: 0.7440 - val_loss: 0.5288
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 71ms/step - accuracy: 0.7526 - loss: 0.5056 - val_accuracy: 0.7270 - val_loss: 0.5629
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 71ms/step - accuracy: 0.7627 - loss: 0.4822 - val_accuracy: 0.7490 - val_loss: 0.5137
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 72ms/step - accuracy: 0.7674 - loss: 0.4765 - val_accuracy: 0.7735 - val_loss: 0.4875
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - accuracy: 0.7853 - loss: 0.4500 - val_accur

## Part 4 - Making a single prediction

In [5]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('datasets/cnn-dataset/single_prediction/pred1.jpg', target_size=(64,64))
test_image = image.img_to_array(test_image)  
test_image = np.expand_dims(test_image, axis=0) # dimension of the batch

result = cnn.predict(test_image)
training_set.class_indices # encoding 1 = dog , 0 = cat
if result[0][0] == 1: # batch then single prediction
    prediction = 'dog'
else: 
    prediction = 'cat'

NameError: name 'cnn' is not defined

In [46]:
print(prediction)

cat


## Export model file

In [17]:
cnn.save('image_classifier.keras')